In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import plotly.io as pio
pio.renderers.default = "notebook"
import plotly.offline as pyo
import plotly.express as px
pyo.init_notebook_mode(connected=True)
import utils as u

In [ ]:
# path = "ray/criteo/bias_varying_epoch_size"
# u.plot_rmsre_boxes(path)

In [ ]:
# u.plot_budget_consumption_boxes(path, epoch_size=7)

In [ ]:
# u.plot_budget_consumption_bars(path, x_axis="num_days_per_epoch")

In [ ]:
# u.plot_budget_consumption_lines(path, epoch_size=7)

In [ ]:
# u.plot_budget_consumption_cdf(path, epoch_size=7)